In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED = dataiku.Dataset("NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED")
NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED_df = NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED.get_dataframe()

In [3]:
NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED_df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,CUSTOMER,ACCOUNTSTATUS,SETUP_DATE,ACCOUNT_CLOSED_DATE,REVENUE_YEAR,REVENUE_QUARTER,REVENUE_MONTH,ACCOUNT_CITY,ACCOUNT_STATE,CUSTOMER_SOURCE_SYSTEM_CODE,CUSTOMER_BUSINESS_PROGRAM_NAME,BI_CUS_PAR,BI_PRODUCT,BI_MRU,REVENUE_AMOUNT_USD,GROSS_SPEND_AMOUNT,PURCHASE_GALLONS_QTY,ACTIVE_CARD_COUNT
0,469007639032,SIEMENS BLDG TECH (2),SIEMENS MEDICAL,Active,2007-12-07 08:00:00+00:00,9999-12-31 00:00:00.000,2019,1,1,Des Plaines,IL,TANDEM,Wheels old,33200,10200,3100,408.592520,18391.35,7602.992,119.0
1,469005701255,CALIFORNIA WATERS,CALIFORNIAWATERS,Active,2017-08-24 07:00:00+00:00,9999-12-31 00:00:00.000,2019,1,1,Des Plaines,IL,TANDEM,Wheels old,33200,10200,3100,2145.328235,114502.95,32432.799,446.0
2,469005701172,HARTUNG BROTHERS,HARTUNG BROTHERS,Terminated,2017-07-03 07:00:00+00:00,2022-07-12 00:00:00.000,2019,1,1,Des Plaines,IL,TANDEM,Wheels old,33200,10200,3100,199.400141,2145.55,616.969,11.0
3,469005700208,CONSTELLATION BRANDS (3CRW),CONSTELLATION BRANDS (3CRW),Terminated,2015-02-18 08:00:00+00:00,2021-03-10 00:00:00.000,2019,1,1,Des Plaines,IL,TANDEM,Wheels old,33200,10200,3100,1139.458388,55804.33,21944.178,344.0
4,469008059644,BALFOUR BEATTY EQUIPMENT ONLY,BALFOUR EQUIP,Active,2009-08-20 07:00:00+00:00,9999-12-31 00:00:00.000,2019,1,1,Des Plaines,IL,TANDEM,Wheels old,33200,10200,3100,701.569042,49047.39,18068.041,110.0


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

#CALCULATED_CARD_DRAW_DOWNS_df = NAFCUSTOMER_REVENUE_BY_CUSTOMER_LIMITED_df # For this sample code, simply copy input to output

# Write recipe outputs
#CALCULATED_CARD_DRAW_DOWNS = dataiku.Dataset("CALCULATED_CARD_DRAW_DOWNS")
#CALCULATED_CARD_DRAW_DOWNS.write_with_schema(CALCULATED_CARD_DRAW_DOWNS_df)